In [ ]:
import fastf1
from matplotlib import pyplot as plt
import fastf1
import fastf1.plotting as f1_plt
import pandas as pd
from datetime import timedelta, datetime
import utils

### session

In [ ]:
session = fastf1.get_session(2022, 21, "Q")
session.load(telemetry=True, laps=True, weather=True)

In [ ]:
session = fastf1.get_session(2022, 21, 'Q')
session.load(laps=True, telemetry=True, weather=True)
laps = session.laps

q1, q2, q3 = laps.split_qualifying_sessions()
wheater = q3.get_weather_data()

fast_mag = q3.pick_driver('MAG')
mag_car_data = fast_mag.get_car_data()
t_mag = mag_car_data['SessionTime']
vCar_mag = mag_car_data['Speed']

fast_ver = q3.pick_driver('VER')
ver_car_data = fast_ver.get_car_data()
t_ver = ver_car_data['SessionTime']
vCar_ver = ver_car_data['Speed']
mag_car_data

In [ ]:
t_mag = mag_car_data['Date']
t_ver = ver_car_data['Date']


# The rest is just plotting
fig, ax = plt.subplots()
ax.plot(t_mag, vCar_mag, label='Mag')
ax.plot(t_ver, vCar_ver, label='Ver')
# ax.plot(wheater['Time'], wheater['WindDirection'], label='Wheater')
ax.set_ylabel('Speed [Km/h]')
ax.set_title('MAG is')
ax.legend()
plt.show()

In [ ]:
session = fastf1.get_session(2022, 21, 'Q')
session.load(laps=True, telemetry=True, weather=True)
laps = session.laps

q1, q2, q3 = laps.split_qualifying_sessions()
wheater = q3.get_weather_data()

fast_mag = q3.pick_driver('MAG')
mag_car_data = fast_mag.get_car_data()
t_mag = mag_car_data['SessionTime']
vCar_mag = mag_car_data['Speed']

fast_ver = q3.pick_driver('VER')
ver_car_data = fast_ver.get_car_data()
t_ver = ver_car_data['SessionTime']
vCar_ver = ver_car_data['Speed']

### Best lap and showing track

In [ ]:
s1_mag = fast_mag.pick_fastest()["Sector1Time"] 
s1_ver = fast_ver.pick_fastest()["Sector1Time"] 
 
s2_mag = fast_mag.pick_fastest()["Sector2Time"] 
s2_ver = fast_ver.pick_fastest()["Sector2Time"] 

s3_mag = fast_mag.pick_fastest()["Sector3Time"] 
s3_ver = fast_ver.pick_fastest()["Sector3Time"] 

s1 = fast_mag.pick_fastest()["Sector1Time"] < fast_ver.pick_fastest()["Sector1Time"] 
s2 = fast_mag.pick_fastest()["Sector2Time"] < fast_ver.pick_fastest()["Sector2Time"] 
s3 = fast_mag.pick_fastest()["Sector3Time"] < fast_ver.pick_fastest()["Sector3Time"] 
print(s1_mag, s1_ver)
print(s2_mag, s2_ver)
print(s3_mag, s3_ver)

print(s1,s2,s3)

In [ ]:
best_lap_mag = fast_mag.pick_lap(25).get_car_data()
best_lap_tel_mag = fast_mag.pick_lap(25).get_telemetry()

best_lap_ver = fast_ver.pick_lap(22).get_car_data()
best_lap_tel_ver = fast_ver.pick_lap(22).get_telemetry()

x_mag = best_lap_tel_mag["X"]
y_mag = best_lap_tel_mag["Y"]
x_ver = best_lap_tel_ver["X"]
y_ver = best_lap_tel_ver["Y"]

img = plt.imread("Brazil_Circuit.png")
fig, ax = plt.subplots()
ax.imshow(img, extent=[-6250, 6850, -2400, 5150])
ax.set_facecolor("white")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

ax.plot(y_mag, -x_mag, label='MAG', c="white", linewidth=1)
plt.show()

### Difference in time

In [ ]:
best_lap_tel_mag["Distance"]

In [ ]:
fig, ax = plt.subplots()
td_0 = timedelta(milliseconds=0)
td_1 = timedelta(milliseconds=100)
for idx in best_lap_tel_mag.index:
    ver = best_lap_tel_ver["Time"][idx]
    mag = best_lap_tel_mag["Time"][idx]
    
    ver_ahead = ver < mag

    ax.scatter(x_mag[idx], y_mag[idx], c='blue'if ver_ahead else 'grey', marker="o")
plt.show()  

### Get differet sectors and mini sectors

In [ ]:
idx, turn_count = utils.find_minisectors(best_lap_tel_mag, turn_difference=800)

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_mag,y_mag)
print(turn_count)
for i in idx:
    ax.scatter(x_mag[i], y_mag[i], c='red')

In [ ]:
idx, first_fastest = utils.compare_minisectors(best_lap_tel_mag, best_lap_tel_ver, turn_difference=800)

In [ ]:
magColor = fastf1.plotting.driver_color("mag")
verColor = fastf1.plotting.driver_color("ver")

In [ ]:
img = plt.imread("Brazil_Circuit.png")
fig, ax = plt.subplots()
ax.imshow(img, extent=[-6250, 6850, -2400, 5150])
ax.set_facecolor("white")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

# ax.plot(x_mag,y_mag, color='black')
for index in x_mag.index:
    if index in idx[:]:
        color = magColor if first_fastest[idx.index(index) - 1] else verColor

    ax.scatter(y_mag[index], -x_mag[index], c=color, s=1)

plt.show()

### weather data

In [ ]:
df_weather = pd.read_csv('q3_wheater.csv', index_col=False)

In [ ]:
df_weather.sort_values("Time", inplace=True)

In [ ]:
best_lap_mag_date_time = best_lap_mag["SessionTime"].apply(utils.convert_delta_to_time)
best_lap_ver_date_time = best_lap_ver["SessionTime"].apply(utils.convert_delta_to_time)

In [ ]:
## track temp over time
fig, ax = plt.subplots()
ax.xaxis.axis_date()
for idx, _  in enumerate(df_weather.index):

    isRainfall = df_weather.iloc[idx]["Rainfall"]
    timeString = df_weather.iloc[idx]["Time"]
    trackTemp = df_weather.iloc[idx]["TrackTemp"]
    timeStamp = utils.convert_string_to_time(timeString)
    ax.plot(timeStamp, trackTemp, c="blue" if isRainfall else "black",  marker="o", markersize=5)
ax.plot(best_lap_mag_date_time, [26]*len(best_lap_mag["SessionTime"]), c="black")
ax.plot(best_lap_ver_date_time, [25.9]*len(best_lap_ver["SessionTime"]), c="blue")